In [ ]:
from PIL import Image, ImageFilter
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
from tqdm import tqdm

FILE_NAME = '9.png'
COLORS = 8
COMPRESSION_PASSES = 1
COMPRESSION_RATE = 0.25
BLUR = True
BLUR_RADIUS = 10
BORDER = False

image = Image.open(f"./source/{FILE_NAME}")  # Replace with your image path
original_image = image
if BLUR:
    image = image.filter(ImageFilter.GaussianBlur(radius=BLUR_RADIUS))


original_matrix = np.array(image)

for _ in tqdm(range(COMPRESSION_PASSES)):
    new_width = int(original_matrix.shape[1]*COMPRESSION_RATE)
    new_height = int(original_matrix.shape[0]*COMPRESSION_RATE)
    image = image.resize(
        (new_width, new_height),
        Image.Resampling.LANCZOS
    )
    image = image.quantize(colors=COLORS, kmeans=3)
    image = image.convert("RGB")
    resized_matrix = np.array(image) 
    image = image.resize((image.width * 2, image.height * 2), Image.NEAREST)
    while image.size[0] < original_matrix.shape[1] and image.size[1] < original_matrix.shape[0]:
        image = image.resize((image.width * 2, image.height * 2), Image.NEAREST)


image_matrix = np.array(image)

if BORDER:
    edge_image = image.convert("L")
    edge_image = edge_image.filter(ImageFilter.EDGE_ENHANCE_MORE)
    edge_image = edge_image.filter(ImageFilter.FIND_EDGES)
    edge_image = edge_image.point(lambda x: 0 if x > 0 else 255)
    edge_image = edge_image.convert("RGB")
    final_matrix_edges = np.array(edge_image)

    mask = (final_matrix_edges == [0, 0, 0])
    image_matrix[mask] = 0


final_image = Image.fromarray(image_matrix)
final_image.save(f"./output/pixelated_{FILE_NAME}", quality=100)   

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(original_image)
axs[0].axis('off')
axs[1].imshow(final_image)
axs[1].axis('off')
axs[0].set_title('Original Image')
axs[1].set_title('Pixelated Image')
plt.tight_layout()
plt.show()